In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [3]:
path = "google-cloud-ncaa-march-madness-2020-division-1-mens-tournament/MDataFiles_Stage1/"
res=pd.read_csv(path+"MRegularSeasonDetailedResults.csv")
tourn = pd.read_csv(path+"MNCAATourneyDetailedResults.csv")

In [4]:
res["Res"]=1
tourn["Res"]=1

In [5]:
features=['Season', 'DayNum', 'WTeamID', 'WScore', 'LTeamID', 'LScore', 'WLoc',
       'NumOT', 'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR',
       'WAst', 'WTO', 'WStl', 'WBlk', 'WPF', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3',
       'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF',
       'Res']
#deal with Wloc categorical
wfeatures=['WScore', 'LScore', 'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR',
       'WAst', 'WTO', 'WStl', 'WBlk', 'WPF', 'Res']
lfeatures=['WScore', 'LScore', 'NumOT', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3',
       'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF',
       'Res']
ffeatures=['WTeamID','WScore', 'LScore', 'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR',
       'WAst', 'WTO', 'WStl', 'WBlk', 'WPF', 'Res']
wfeatures2=["Team1WScore", 'WFGM', 'WFGA', 'WFGM3',
       'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR', 'WAst', 'WTO', 'WStl', 'WBlk',
       'WPF']
lfeatures2=['Team1LScore', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3', 'LFTM', 'LFTA', 'LOR', 'LDR',
       'LAst', 'LTO', 'LStl', 'LBlk', 'LPF']


In [6]:
columns=["Points", "FG", "FGA", "FG3", "FGA3", "FT",
 "FTA", "OR", "DR", "Ast", "To", "Stl", "Blk", "Pf" ]

teams=set(np.append(res.WTeamID.unique(), res.LTeamID.unique()))
seasons=res.Season.unique()
def cleanData(x):
    final = pd.DataFrame()
    #transform and clean dataframe x
    for season in seasons:
        for team in teams:
            win, loss = subset(x, season, team)
            if(win.empty==True or loss.empty==True):
                #defeated
                if (win.empty==True):
                    merge = defeated(loss)
                #undefeated
                elif (loss.empty==True):
                    merge = undefeated(win)
            else:
                winTot, lossTot = group(win, loss, wfeatures, lfeatures)
            #prob need to see if winTot gives you an empty dataframe or not
                merge = mergeData(winTot, lossTot, ffeatures, columns)
                merge=calcGame(merge, columns,wfeatures2, lfeatures2)
            #merge everything
            final = pd.concat([final, merge])
            
    return final

def subset(x, b, c):
    # a is dataframe, b is Season, c is TeamID
    win = x[(x["Season"]==b) & (x["WTeamID"]==c)] 
    loss = x[(x["Season"]==b) & (x["LTeamID"]==c)]
    return win, loss

def undefeated(df):
    #print('yu')
    new = df.groupby(["Season", "WTeamID"])[wfeatures].sum()
    new = new.reset_index()
    new=new.rename(columns={'WTeamID': 'TeamID', 'WScore': 'Points', 'Res': 'Wins', 'WFGM': 'FG', 'WFGA': 'FGA',
                        'WFGM3': 'FG3', 'WFGA3': 'FGA3', 'WFTM': 'FT', 'WFTA': 'FTA',
                        'WOR': 'OR', 'WDR':'DR', 'WAst': 'Ast', 'WTO':'To', 'WStl': 'Stl',
                        'WBlk': 'Blk', 'WPF': 'Pf'})
    #res_y is number of losses
    new["Losses"]=0
    new["games"] =new["Wins"]
    new["totalMargin"] = new["Points"]-new["LScore"]
    
    new=new[(["TeamID", "Season", "games", "totalMargin", "Wins", "Losses"]+columns)]
    for i in range(0, len(columns)):
        new.loc[:,columns[i]] = new.loc[:,columns[i]]/new["games"]
    new.loc[:, "totalMargin"] = new.loc[:,"totalMargin"]/new["games"]
    print 
    return new 

def defeated(df):
    #if team is undefeated or defeated
    new = df.groupby(["Season", "LTeamID"])[lfeatures].sum()
    new = new.reset_index()
    #res_x is number of wins
    
    new=new.rename(columns={'LTeamID':'TeamID', 'LScore': 'Points', 'Res': 'Losses', 'LFGM': 'FG', 'LFGA': 'FGA',
                        'LFGM3': 'FG3', 'LFGA3': 'FGA3', 'LFTM': 'FT', 'LFTA': 'FTA',
                        'LOR': 'OR', 'LDR':'DR', 'LAst': 'Ast', 'LTO':'To', 'LStl': 'Stl',
                        'LBlk': 'Blk', 'LPF': 'Pf'})
    new["Wins"]=0
    new["games"]=new["Losses"]
    
    new["totalMargin"] = new["Points"]-new["WScore"]
    
    new=new[(["TeamID", "Season", "games", "totalMargin", "Wins", "Losses"]+columns)]
    for i in range(0, len(columns)):
        new.loc[:,columns[i]] = new.loc[:,columns[i]]/new["games"]
    new.loc[:, "totalMargin"] = new.loc[:,"totalMargin"]/new["games"]
    return new

def group(win, loss, wfeatures, lfeatures):
    winTot=win.groupby(["Season", "WTeamID"])[wfeatures].sum()
    lossTot=loss.groupby(["Season", "LTeamID"])[lfeatures].sum()
    winTot=winTot.reset_index()
    lossTot=lossTot.reset_index()
    return winTot, lossTot

def mergeData(winTot, lossTot, ffeatures, columns):
    #merge wins and losses
    merge=pd.merge(winTot[ffeatures], lossTot, left_on="WTeamID", right_on="LTeamID")
    merge["games"]=merge["Res_x"]+merge["Res_y"]
    merge=merge.rename(columns={'WScore_x': 'Team1WScore', 'LScore_x': 'Team2LScore', 'LScore_y': 'Team1LScore', 'WScore_y': 'Team2WScore', 'Res_x':'Wins', 'Res_y': 'Losses'})
    merge["WMargin"]=merge["Team1WScore"] - merge["Team2LScore"]
    merge["LMargin"]=merge["Team1LScore"] - merge["Team2WScore"]
    merge["totalMargin"] = merge["WMargin"] + merge["LMargin"]
    return merge

def calcGame(merge, columns, wfeatures2, lfeatures2):
    for i in range(0,len(wfeatures2)):
        merge.loc[:,columns[i]]=merge.loc[:,wfeatures2[i]] + merge.loc[:,lfeatures2[i]]
    merge=merge.rename(columns={"WTeamID": "TeamID"})
    merge=merge[(["TeamID", "Season", "games", "totalMargin", "Wins", "Losses"]+columns)]
    for i in range(0, len(columns)):
        merge.loc[:,columns[i]] = merge.loc[:,columns[i]]/merge["games"]
    merge.loc[:, "totalMargin"] = merge.loc[:,"totalMargin"]/merge["games"]
    
    return merge

In [7]:
final = cleanData(res)
final.head()

,TeamID,Season,games,totalMargin,Wins,Losses,Points,FG,FGA,FG3,FGA3,FT,FTA,OR,DR,Ast,To,Stl,Blk,Pf
0,1102.0,2003.0,28,0.250000,12,16,57.250000,19.142857,39.785714,7.821429,20.821429,11.142857,17.107143,4.178571,16.821429,13.000000,11.428571,5.964286,1.785714,18.750000
0,1103.0,2003.0,27,0.629630,13,14,78.777778,27.148148,55.851852,5.444444,16.074074,19.037037,25.851852,9.777778,19.925926,15.222222,12.629630,7.259259,2.333333,19.851852
0,1104.0,2003.0,28,4.285714,17,11,69.285714,24.035714,57.178571,6.357143,19.857143,14.857143,20.928571,13.571429,23.928571,12.107143,13.285714,6.607143,3.785714,18.035714
0,1105.0,2003.0,26,-4.884615,7,19,71.769231,24.384615,61.615385,7.576923,20.769231,15.423077,21.846154,13.500000,23.115385,14.538462,18.653846,9.307692,2.076923,20.230769
0,1106.0,2003.0,28,-0.142857,13,15,63.607143,23.428571,55.285714,6.107143,17.642857,10.642857,16.464286,12.285714,23.857143,11.678571,17.035714,8.357143,3.142857,18.178571


In [8]:
final.to_csv("nba.csv")